In [2]:
%pwd

'd:\\Projects\\uob-project\\src\\sub'

In [3]:
# RUN ONLY ONCE!
%cd ../../

d:\Projects\uob-project


In [7]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

EXPECTED_NEWS_PER_DAY = 3
NEWS_EMBEDDINGS_PATH = "data/interim/news_reduced_3_per_day_embeddings.csv"
BITCOIN_PATH = "data/interim/bitcoin.csv"
DONT_PREDICT = ["change in next day (%)", "DayAfterTmr"]

In [4]:
df_bitcoin_raw = pd.read_csv(BITCOIN_PATH)
news = pd.read_csv(NEWS_EMBEDDINGS_PATH)
news.head()

,Date,Headline,similarity,search
0,2019-01-01,Darwin rugby league player uses bitcoin to buy...,"[-0.01071454118937254, 0.011403998360037804, 0...","[-0.014798003248870373, 0.00944652408361435, 0..."
1,2019-01-01,"While Bitcoin ‘Died’ 90 Times In 2018, There A...","[-0.008313984610140324, 0.015355568379163742, ...","[-0.01103475783020258, 0.0016785092884674668, ..."
2,2019-01-01,"Crypto Market Update Jan.1: New Year, New Hope...","[-0.0011497164377942681, -0.005909790750592947...","[-0.005611605476588011, -0.025839712470769882,..."
3,2019-01-02,Hackers Threaten to Dump 9/11 Related Insuranc...,"[0.0009691137238405645, 0.006849941331893206, ...","[0.0014824277022853494, -0.0005108365439809859..."
4,2019-01-02,Marketing Was A Driving Force In The ICO Hype ...,"[-0.0020507564768195152, -0.001764271291904151...","[-0.014102131128311157, -0.011523179709911346,..."


In [39]:
def repeat_if_less(df, expected_len=EXPECTED_NEWS_PER_DAY):
    n_repeats = expected_len - len(df)

    if n_repeats > 0:
        extra_rows = [df.iloc[-1].values.tolist()] * n_repeats
        extra_rows = pd.DataFrame(extra_rows, columns=df.columns)
        df = pd.concat([df, extra_rows], ignore_index=True)
    
    return df

news_temp = news.groupby("Date").apply(repeat_if_less).reset_index(drop=True)
news_temp.drop(["Headline"], axis=1, inplace=True)
news_temp.columns = ["date", "similarity", "search"]
news_temp.set_index("date", inplace=True)
news_temp.head()

# news_temp.to_csv("data/processed/sentimental_3_per_day.csv")

,Date,Headline,similarity,search
0,2019-01-01,Darwin rugby league player uses bitcoin to buy...,"[-0.01071454118937254, 0.011403998360037804, 0...","[-0.014798003248870373, 0.00944652408361435, 0..."
1,2019-01-01,"While Bitcoin ‘Died’ 90 Times In 2018, There A...","[-0.008313984610140324, 0.015355568379163742, ...","[-0.01103475783020258, 0.0016785092884674668, ..."
2,2019-01-01,"Crypto Market Update Jan.1: New Year, New Hope...","[-0.0011497164377942681, -0.005909790750592947...","[-0.005611605476588011, -0.025839712470769882,..."
3,2019-01-02,Hackers Threaten to Dump 9/11 Related Insuranc...,"[0.0009691137238405645, 0.006849941331893206, ...","[0.0014824277022853494, -0.0005108365439809859..."
4,2019-01-02,Marketing Was A Driving Force In The ICO Hype ...,"[-0.0020507564768195152, -0.001764271291904151...","[-0.014102131128311157, -0.011523179709911346,..."


In [8]:
# Prepare 1st dataframe phase for bitcoin data
df_bitcoin = df_bitcoin_raw.sort_values("Date")
df_bitcoin.index = pd.to_datetime(df_bitcoin["Date"].apply(lambda x: x.replace('/', '-')))
df_bitcoin.drop(DONT_PREDICT + ["Date"], axis=1, inplace=True)

# Renaming
col_names = list(map(str.lower, df_bitcoin.columns))
col_names[-2] = "change"
col_names[-1] = "next"
df_bitcoin.columns = col_names
df_bitcoin.index.name = "date"

df_bitcoin.head()
df_bitcoin.to_csv("data/processed/bitcoin.csv")